### N Gram Analysis According To Fourgram

In [58]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [59]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [60]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index      

In [61]:
ngram_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/1-N Gram Data Prepare"

ngram_used_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/N Gram Used"


path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/2-N Gram Analysis According To Fourgram"

Path(path).mkdir(parents=True, exist_ok=True)

In [62]:
def strip(df,column_list):
    '''
    strip(df,column_list) df is dataframe and column_list are selected columns \n
    strip(df_twogram_used,df_twogram_used.columns)
    '''
    for i in column_list:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [63]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [64]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed  
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [65]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Ngram Read

In [66]:
df_twogram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Twogram_Used.xlsx")
df_twogram_used = strip(df_twogram_used,df_twogram_used.columns)
df_twogram_used

,twogram
0,bir şey
1,bu kadar
2,ne kadar
3,ve de
4,ne için
...,...
194,senin için
195,sorun değil
196,sorun yok
197,tamam mı


In [67]:
df_threegram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Threegram_Used.xlsx")
df_threegram_used = strip(df_threegram_used, df_threegram_used.columns)
df_threegram_used

,threegram
0,ama neden böyle
1,ama o benim
2,ama şimdi olmaz
3,artık herkes biliyor
4,aynı şekilde devam
...,...
343,bir de şu
344,bir daha asla
345,peki ne için
346,daha da fazla


In [68]:
twogram_used_list = list(set(df_twogram_used["twogram"]))
threegram_used_list = list(set(df_threegram_used["threegram"]))

In [69]:
df_twogram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Twogram_Selected.csv")
df_twogram_remove_used = df_twogram[~df_twogram["twogram"].isin(twogram_used_list)]
df_twogram_remove_used

,twogram,frequency
6,hayır hayır,266457
9,özür dilerim,260661
11,ne yapıyorsun,236176
15,hiçbir şey,209701
17,ne demek,188846
...,...,...
220163,dışarıda bizimle,6
220164,bizim kimseye,6
220165,tarafa bunu,6
220166,beri kahrolası,6


In [70]:
df_threegram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Threegram_Selected.csv")
df_threegram_remove_used = df_threegram[~df_threegram["threegram"].isin(threegram_used_list)]
df_threegram_remove_used

,threegram,frequency
3,her şey yolunda,75968
5,hayır hayır hayır,69443
9,ne demek istiyorsun,63142
10,sana bir şey,56713
11,böyle bir şey,51970
...,...,...
500343,sonra insanlar bu,10
500344,o anda dünya,10
500345,zaman zaman sizinle,10
500346,bir güç sahibi,10


In [71]:
df_fourgram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Fourgram_Selected.csv")
df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
366580,sadece yemek ve içki,7
366581,küçük bir şey ve,7
366582,olduğunu bu yüzden biliyorum,7
366583,sadece yeri değil zamanı,7


In [72]:
df_fourgram_selected_for_course = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/\
Data/Manuel Select/Selected 4grams.xlsx")
df_fourgram_selected_for_course

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
95,her şey iyi olacak,2479
96,öyle bir şey değil,2478
97,bunu sen mi yaptın,2453
98,ne demek oluyor bu,2431


#### Ngram In Ngram

In [73]:
twogram_list = df_twogram_remove_used["twogram"].to_list()  # without used
threegram_list = df_threegram_remove_used["threegram"].to_list()  # without used
fourgram_list = df_fourgram["fourgram"].to_list()
fourgram_course_list = df_fourgram_selected_for_course["fourgram"].to_list()

##### Twogram Used In Threegram Used (3A)

In [74]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_used

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [75]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_used_in_three_used = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_used_in_three_used

,twogram,threegram
0,onun için,onun için de
1,onun için,onun için çok
2,onun için,onun için mi
3,küçük kız,bir küçük kız
4,küçük kız,bu küçük kız
...,...,...
376,senin için,senin için çok
377,başka biri,başka biri için
378,başka biri,başka biri mi
379,başka biri,başka biri var


In [ ]:
#df_two_used_in_threegram_used_list = df_two_used_in_three_used.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
#df_two_used_in_threegram_used_list

##### Twogram Used In Threegram Used In Fourgram (1A)

In [76]:
threegram_include_twogram_list = list(set(df_two_used_in_three_used["threegram"]))

In [77]:
# parameter
source_wordgroup_list = threegram_include_twogram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [78]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_twogram_used_in_three_used = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_twogram_used_in_three_used

,threegram,fourgram
0,biliyorsun değil mi,bunu biliyorsun değil mi
1,bu kadar uzun,neden bu kadar uzun
2,bir şey yok,diye bir şey yok
3,ol tamam mı,sakin ol tamam mı
4,bir şey yok,başka bir şey yok
5,bir şey yok,bir şey yok mu
6,bir şey yok,hiç bir şey yok
7,bir şey yok,öyle bir şey yok
8,bir şey yok,yapacak bir şey yok
9,bir şey yok,pek bir şey yok


In [81]:
df_twogram_used_in_three_used_in_selected_four = pd.merge(df_two_used_in_three_used, df_twogram_used_in_three_used, how="inner", on="threegram")
df_twogram_used_in_three_used_in_selected_four.drop_duplicates(inplace=True)
df_twogram_used_in_three_used_in_selected_four

,twogram,threegram,fourgram
0,biliyor musun,var biliyor musun,ne var biliyor musun
1,evet ben,evet ben de,evet ben de öyle
3,ben de,evet ben de,evet ben de öyle
5,sen de,sen de biliyorsun,bunu sen de biliyorsun
6,bu kadar,bu kadar çok,neden bu kadar çok
7,bu kadar,neden bu kadar,neden bu kadar uzun
8,bu kadar,neden bu kadar,neden bu kadar çok
9,bu kadar,bu kadar mı,hepsi bu kadar mı
10,teşekkür ederim,çok teşekkür ederim,için çok teşekkür ederim
11,bu kadar,bu kadar uzun,neden bu kadar uzun


In [ ]:
#df_twogram_used_in_three_used_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Threegram_Used_In_Selected_Fourgram_Result.xlsx", index=False)  # (1A)

In [89]:
three_in_selected_four_list = list(set(df_twogram_used_in_three_used_in_selected_four["threegram"]))

In [90]:
df_two_used_in_three_used_not_in_four = df_two_used_in_three_used[~df_two_used_in_three_used["threegram"].isin(three_in_selected_four_list)]
df_two_used_in_three_used_not_in_four

,twogram,threegram
0,onun için,onun için de
1,onun için,onun için çok
2,onun için,onun için mi
3,küçük kız,bir küçük kız
4,küçük kız,bu küçük kız
...,...,...
376,senin için,senin için çok
377,başka biri,başka biri için
378,başka biri,başka biri mi
379,başka biri,başka biri var


In [ ]:
#df_two_used_in_three_used_not_in_four.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Threegram_Used_Result.xlsx", index=False)  # 3A 

##### Twogram Used In Selected Fourgram (2A)

In [95]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [96]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_used_in_selected_four

,twogram,fourgram
0,biliyor musun,ne var biliyor musun
1,biliyor musun,ne olduğunu biliyor musun
2,biliyor musun,nerede olduğunu biliyor musun
3,biliyor musun,ne düşünüyorum biliyor musun
4,biliyor musun,ne demek biliyor musun
...,...,...
81,bir şey,bir şey duydun mu
82,bir şey,kötü bir şey mi
83,bir şey,bir şey söyleyeyim mi
84,bir şey,öyle bir şey olmayacak


In [ ]:
#df_two_used_in_selected_four_list = df_two_used_in_selected_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_two_used_in_selected_four_list

In [103]:
df_test = df_two_used_in_selected_four[~df_two_used_in_selected_four["fourgram"].isin(list(set(df_twogram_used_in_three_used_in_selected_four["fourgram"])))]
df_test

,twogram,fourgram
1,biliyor musun,ne olduğunu biliyor musun
2,biliyor musun,nerede olduğunu biliyor musun
3,biliyor musun,ne düşünüyorum biliyor musun
4,biliyor musun,ne demek biliyor musun
5,her şeyi,için her şeyi yaparım
6,her şey,her şey yolunda mı
7,her şey,her şey kontrol altında
8,her şey,her şey için teşekkürler
13,ister misin,bir şey ister misin
14,ya da,öyle ya da böyle


In [104]:
df_test[~df_test["twogram"].isin(list(set(df_two_used_in_three_used_not_in_four["twogram"])))]

,twogram,fourgram
6,her şey,her şey yolunda mı
7,her şey,her şey kontrol altında
8,her şey,her şey için teşekkürler
25,hayır mı,evet mi hayır mı
44,evet mi,evet mi hayır mı
45,sen mi,bunu sen mi yaptın


In [ ]:
#df_two_used_in_selected_four_list.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Selected_Fourgram_Result.xlsx", index=False)

##### Threegram Used In Selected Fourgram (4A)

In [100]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [101]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_used_in_selected_four

,threegram,fourgram
0,bir şey daha,bir şey daha var
1,bir şey daha,başka bir şey daha
2,bir şey yok,diye bir şey yok
3,bir şey yok,başka bir şey yok
4,bir şey yok,bir şey yok mu
5,biliyorsun değil mi,bunu biliyorsun değil mi
6,bir şey yok,hiç bir şey yok
7,bir şey yok,öyle bir şey yok
8,bir şey yok,yapacak bir şey yok
9,bir şey yok,pek bir şey yok


In [105]:
df_test2 = df_three_used_in_selected_four[~df_three_used_in_selected_four["fourgram"].isin(list(set(df_twogram_used_in_three_used_in_selected_four["fourgram"])))]
df_test2

,threegram,fourgram


In [106]:
df_test2[~df_test2["threegram"].isin(list(set(df_two_used_in_three_used_not_in_four["threegram"])))]

,threegram,fourgram


In [ ]:
#df_three_used_in_selected_four_list = df_three_used_in_selected_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_three_used_in_selected_four_list

In [ ]:
#df_three_used_in_selected_four_list.to_excel(f"{lang_folder.capitalize()}_Threegram_Used_In_Selected_Fourgram_Result.xlsx", index=False)

##### Twogram In Threegram (3B)

In [14]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three

In [ ]:
df_two_in_threegram_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_threegram_list

In [ ]:
#df_two_in_threegram_list.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_Result.xlsx", index=False)

##### Twogram In Selected Fourgram (2B)

In [14]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [15]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_selected_four

,twogram,fourgram
0,bir şey,bir şey var mı
1,bir şey,gereken bir şey var
2,bir şey,başka bir şey var
3,bir şey,bir şey daha var
4,bir şey,bir şey mi var
...,...,...
295,mi hayır,evet mi hayır mı
296,yapıyorsun değil,şaka yapıyorsun değil mi
297,şey buldun,bir şey buldun mu
298,öyle ya,öyle ya da böyle


In [16]:
df_two_in_selected_four_list = df_two_in_selected_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_selected_four_list

,twogram,fourgram
0,bana yardım,[bana yardım eder misin]
1,başka bir,"[başka bir şey var, başka bir şey yok, başka b..."
2,ben de,"[ben de seni seviyorum, evet ben de öyle]"
3,benim için,[benim için çok önemli]
4,biliyor musun,"[ne var biliyor musun, ne olduğunu biliyor mus..."
...,...,...
185,şey yolunda,[her şey yolunda mı]
186,şeye ihtiyacın,[bir şeye ihtiyacın olursa]
187,şeyi yaparım,[için her şeyi yaparım]
188,şeyin var,[bir şeyin var mı]


In [17]:
#df_two_in_selected_four_list.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Selected_Fourgram_Result.xlsx", index=False)

##### Threegram In Selected Fourgram (4B)

In [15]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [16]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_selected_four

,threegram,fourgram
0,bir şey yok,diye bir şey yok
1,bir şey yok,başka bir şey yok
2,bir şey yok,bir şey yok mu
3,bir şey yok,hiç bir şey yok
4,bir şey yok,öyle bir şey yok
...,...,...
195,sakin ol tamam,sakin ol tamam mı
196,istiyor musun istemiyor,istiyor musun istemiyor musun
197,şey söyleyeyim mi,bir şey söyleyeyim mi
198,olur ne olmaz,ne olur ne olmaz


In [17]:
df_three_in_selected_four_list = df_three_in_selected_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_selected_four_list

,threegram,fourgram
0,bana yardım eder,[bana yardım eder misin]
1,başka bir şey,"[başka bir şey var, başka bir şey yok, başka b..."
2,ben de seni,[ben de seni seviyorum]
3,ben de öyle,[evet ben de öyle]
4,benim için çok,[benim için çok önemli]
...,...,...
158,şey yok mu,[bir şey yok mu]
159,şey yolunda mı,[her şey yolunda mı]
160,şeye ihtiyacın olursa,[bir şeye ihtiyacın olursa]
161,şeyin var mı,[bir şeyin var mı]


In [21]:
#df_three_in_selected_four_list.to_excel(f"{lang_folder.capitalize()}_Threegram_In_Selected_Fourgram_Result.xlsx", index=False)

##### Twogram In Threegram In Selected Fourgram (1B)

In [18]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_three_in_selected_four_list

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [19]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three

,twogram,threegram
0,bir şey,başka bir şey
1,bir şey,bir şey buldun
2,bir şey,bir şey daha
3,bir şey,bir şey değil
4,bir şey,bir şey duydun
...,...,...
321,şey buldun,bir şey buldun
322,şey buldun,şey buldun mu
323,öyle ya,öyle ya da
324,şey kontrol,her şey kontrol


In [20]:
df_twogram_in_three_in_selected_four = pd.merge(df_two_in_three, df_three_in_selected_four_list, how="inner", on="threegram")
df_twogram_in_three_in_selected_four

,twogram,threegram,fourgram
0,bir şey,başka bir şey,"[başka bir şey var, başka bir şey yok, başka b..."
1,başka bir,başka bir şey,"[başka bir şey var, başka bir şey yok, başka b..."
2,bir şey,bir şey buldun,[bir şey buldun mu]
3,şey buldun,bir şey buldun,[bir şey buldun mu]
4,bir şey,bir şey daha,"[bir şey daha var, başka bir şey daha]"
...,...,...,...
321,şey kontrol,şey kontrol altında,[her şey kontrol altında]
322,ihtiyacın olursa,şeye ihtiyacın olursa,[bir şeye ihtiyacın olursa]
323,şeye ihtiyacın,şeye ihtiyacın olursa,[bir şeye ihtiyacın olursa]
324,ne düşünüyorum,ne düşünüyorum biliyor,[ne düşünüyorum biliyor musun]


In [21]:
df_twogram_in_three_in_selected_four = pd.merge(df_two_in_three, df_three_in_selected_four, how="inner", on="threegram")
df_twogram_in_three_in_selected_four

,twogram,threegram,fourgram
0,bir şey,başka bir şey,başka bir şey var
1,bir şey,başka bir şey,başka bir şey yok
2,bir şey,başka bir şey,başka bir şey değil
3,bir şey,başka bir şey,başka bir şey daha
4,başka bir,başka bir şey,başka bir şey var
...,...,...,...
395,şey kontrol,şey kontrol altında,her şey kontrol altında
396,ihtiyacın olursa,şeye ihtiyacın olursa,bir şeye ihtiyacın olursa
397,şeye ihtiyacın,şeye ihtiyacın olursa,bir şeye ihtiyacın olursa
398,ne düşünüyorum,ne düşünüyorum biliyor,ne düşünüyorum biliyor musun


In [ ]:
#df_twogram_in_three_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_In_Selected_Fourgram_Result.xlsx", index=False)

#### Copy Move And Delete

In [114]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['Turkish_Word_Text_Selection_Result.csv',
 'Turkish_Ninegram_Text_Selection_Result.csv',
 'Turkish_Eightgram_Text_Selection_Result.csv',
 'Turkish_Sevengram_Text_Selection_Result.csv']

In [115]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [116]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass